In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import DenseVector

spark = SparkSession.builder.master("local") \
                            .appName("appMLiris") \
                            .config("spark.executor.memory", "1gb") \
                            .getOrCreate()


22/04/29 03:21:39 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.228.131 instead (on interface ens33)
22/04/29 03:21:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/29 03:21:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#rdd = sc.textFile('/Users/Storage/Vicohub/Data/Iris.csv’)
#df = spark.read.csv('/Users/Storage/Vicohub/Data/Iris.csv', header=True)
df = spark.read.csv('hdfs://localhost:9000/iris_2.csv', header=True)
input_data = df.rdd.map(lambda x: (DenseVector(x[:4]), x[4]))
df1 = spark.createDataFrame(input_data, ["X", "Y"])

22/04/29 03:29:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
 Schema: _c0, Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/iris_2.csv


In [6]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
indexer = StringIndexer(inputCol="Y", outputCol="Yn")
indexed = indexer.fit(df1).transform(df1)
(trainingData, testData) = indexed.randomSplit([0.7, 0.3])
rf = RandomForestClassifier(labelCol="Yn",
featuresCol="X",
numTrees=10)
model = rf.fit(trainingData)
predictions = model.transform(testData)

22/04/29 03:29:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
 Schema: _c0, Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/iris_2.csv
22/04/29 03:29:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
 Schema: _c0, Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/iris_2.csv
22/04/29 03:29:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
 Schema: _c0, Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/iris_2.csv
22/04/29 03:29:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ,

In [7]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
predictions.select("Yn", "prediction")
evaluator = MulticlassClassificationEvaluator(
labelCol="Yn", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g" % (accuracy))

Test Accuracy = 0.381818


22/04/29 03:30:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
 Schema: _c0, Sepal.Length, Sepal.Width, Petal.Length, Petal.Width, Species
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/iris_2.csv
